# Flood Height to Raster Processor

This code takes flood heights and DEMs as inputs and uses the ArcGIS Con function and MosaictoRaster function to find the inundated areas for each division and output them as a binary raster of flooded or not flooded for each division and the entire area of interest.

## Import Statements

In [1]:
import arcpy
from arcpy import env
from arcpy.sa import *
import pandas as pd
import os

## Flood Heights without Protection

### Import csvs

In [2]:
flood_heights_c = pd.read_csv("Sample_Output_Data\\single_storm_flood_heights_c.csv",header=None).values
flood_heights_w = pd.read_csv("Sample_Output_Data\\single_storm_flood_heights_w.csv",header=None).values

### Create rasters

In [3]:
#Warm Storm

# location of original, divisioned, DEM files
env.workspace = r"Data\LM_div18"
env.overwriteOutput = True

rasters=[]

divfolder = r"IntermediateData\div_Rasters"
if not os.path.exists(divfolder):
    os.makedirs(divfolder)
rasterfolder = r"Sample_Output_Data\Flood_Rasters_w"
if not os.path.exists(rasterfolder):
    os.makedirs(rasterfolder)

#Processing Warm Storms
for i in range(len(flood_heights_w)):
    raster_name=r"Data\LM_div18\dem_lm_z35_%d.TIF"%i
    inRaster = Raster(raster_name)
    whereClause="VALUE<%f"%flood_heights_w[0][i]
    outCon = Con(inRaster, 1, 0, whereClause)
    outPath = r"IntermediateData\div_Rasters\storm_cold_div%d.TIF"%i
    outCon.save(outPath)
    rasters.append(outPath)

arcpy.MosaicToNewRaster_management(rasters, rasterfolder , "Inundation_areas_w.tif",None,None,None,1,None,None)

<Result 'Sample_Output_Data\\Flood_Rasters_w\\Inundation_areas_w.tif'>

In [4]:
#Cold Storm

# location of original, divisioned, DEM files
env.workspace = r"Data\LM_div18"
env.overwriteOutput = True

rasters=[]

divfolder = r"IntermediateData\div_Rasters"
if not os.path.exists(divfolder):
    os.makedirs(divfolder)
rasterfolder = r"Sample_Output_Data\Flood_Rasters_c"
if not os.path.exists(rasterfolder):
    os.makedirs(rasterfolder)

#Processing Warm Storms
for i in range(len(flood_heights_c)):
    raster_name=r"Data\LM_div18\dem_lm_z35_%d.TIF"%i
    inRaster = Raster(raster_name)
    whereClause="VALUE<%f"%flood_heights_c[0][i]
    outCon = Con(inRaster, 1, 0, whereClause)
    outPath = r"IntermediateData\div_Rasters\storm_cold_div%d.TIF"%i
    outCon.save(outPath)
    rasters.append(outPath)

arcpy.MosaicToNewRaster_management(rasters, rasterfolder, "Inundation_areas_c.tif",None,None,None,1,None,None)

<Result 'Sample_Output_Data\\Flood_Rasters_c\\Inundation_areas_c.tif'>

## Flood Heights with Protection

### Import csvs

In [5]:
flood_heights_c = pd.read_csv("Sample_Output_Data\\single_storm_flood_heights_c.csv",header=None).values
flood_heights_w = pd.read_csv("Sample_Output_Data\\single_storm_flood_heights_w.csv",header=None).values

### Create rasters

In [6]:
#Warm Storm

# location of original, divisioned, DEM files
env.workspace = r"Data\LM_div18"
env.overwriteOutput = True

rasters=[]

divfolder = r"IntermediateData\div_Rasters"
if not os.path.exists(divfolder):
    os.makedirs(divfolder)
rasterfolder = r"Sample_Output_Data\Flood_Rasters_c"
if not os.path.exists(rasterfolder):
    os.makedirs(rasterfolder)

#Processing Warm Storms
for i in range(len(flood_heights_c)):
    raster_name=r"Data\LM_div18\dem_lm_z35_%d.TIF"%i
    inRaster = Raster(raster_name)
    whereClause="VALUE<%f"%flood_heights_w[0][i]
    outCon = Con(inRaster, 1, 0, whereClause)
    outPath = r"IntermediateData\div_Rasters\storm_warm_div%d.TIF"%i
    outCon.save(outPath)
    rasters.append(outPath)

arcpy.MosaicToNewRaster_management(rasters, rasterfolder, "Inundation_areas_w.tif",None,None,None,1,None,None)

<Result 'Sample_Output_Data\\Flood_Rasters_c\\Inundation_areas_w.tif'>

In [7]:
#Cold Storm

# location of original, divisioned, DEM files
env.workspace = r"Data\LM_div18"
env.overwriteOutput = True

rasters=[]

divfolder = r"IntermediateData\div_Rasters"
if not os.path.exists(divfolder):
    os.makedirs(divfolder)
rasterfolder = r"Sample_Output_Data\Flood_Rasters_c"
if not os.path.exists(rasterfolder):
    os.makedirs(rasterfolder)

#Processing Warm Storms
for i in range(len(flood_heights_c)):
    raster_name=r"Data\LM_div18\dem_lm_z35_%d.TIF"%i
    inRaster = Raster(raster_name)
    whereClause="VALUE<%f"%flood_heights_c[0][i]
    outCon = Con(inRaster, 1, 0, whereClause)
    outPath = r"IntermediateData\div_Rasters\storm_cold_div%d.TIF"%i
    outCon.save(outPath)
    rasters.append(outPath)

arcpy.MosaicToNewRaster_management(rasters, rasterfolder, "Inundation_areas_c.tif",None,None,None,1,None,None)

<Result 'Sample_Output_Data\\Flood_Rasters_c\\Inundation_areas_c.tif'>